In [30]:
!pip -q install SQLAlchemy psycopg2-binary pandas


In [31]:
# Simplified query cell (uses session-level search_path)
# - Connect with the STUDENT URL
# - Set search_path so we can omit schema prefixes
# - Run example queries and show results

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Example 1: Large-Screen Laptops (≥15\" and under $2000)
with engine.connect() as conn:
    laptop_sql = """
    SELECT p.maker, l.model, l.screen, l.price
    FROM product p
    JOIN laptop l ON p.model = l.model
    WHERE l.screen >= 15 AND l.price < 2000
    ORDER BY l.price ASC;
    """
    laptop_df = pd.read_sql_query(text(laptop_sql), conn)

# Display results
print("Large-Screen Laptops (≥15\" and under $2000):")
display(laptop_df)


# Example 2: Makers with the Most PC Models
with engine.connect() as conn:
    pc_sql = """
    SELECT p.maker, COUNT(*) AS num_pcs
    FROM product p
    JOIN pc ON p.model = pc.model
    GROUP BY p.maker
    ORDER BY num_pcs DESC;
    """
    pc_df = pd.read_sql_query(text(pc_sql), conn)

# Display results
print("Makers with the Most PC Models:")
display(pc_df)


# Example 3: Average Printer Price by Type
with engine.connect() as conn:
    printer_sql = """
    SELECT pr.type,
           ROUND(AVG(pr.price), 2) AS avg_price,
           COUNT(*) AS num_models,
           MIN(pr.price) AS cheapest,
           MAX(pr.price) AS most_expensive
    FROM printer pr
    GROUP BY pr.type
    ORDER BY avg_price ASC;
    """
    printer_df = pd.read_sql_query(text(printer_sql), conn)

# Display results
print("Average Printer Price by Type:")
display(printer_df)


Paste STUDENT DB URL (hidden): ··········
Large-Screen Laptops (≥15" and under $2000):


,maker,model,screen,price
0,E,2003,16.4,549
1,F,2008,15.4,900
2,E,2002,17.0,949
3,A,2006,15.4,1700


Makers with the Most PC Models:


,maker,num_pcs
0,B,3
1,D,3
2,E,3
3,A,3
4,C,1


Average Printer Price by Type:


,type,avg_price,num_models,cheapest,most_expensive
0,ink-jet,106.33,3,99,120
1,laser,364.50,4,120,899
